<a href="https://colab.research.google.com/github/amrutha2413/NN-for-detect/blob/main/NN_for_detecting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -----------------------------
# MNIST DIGIT CLASSIFIER (PyTorch)
# -----------------------------

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
# -----------------------------
# 1. LOAD DATA
# Transforms are preprocessing steps that get applied automatically to every image
# you load from a dataset.
# Think of transforms as a recipe that says:

# “Every time you give me an image, do X, then Y, then Z to it.”
# “For every MNIST image: convert it to a PyTorch tensor.
# MNIST images come in as PIL images (Python Imaging Library).

# But your neural network expects tensors.
# -----------------------------
transform = transforms.Compose([
    transforms.ToTensor()
])

In [4]:
# Load training dataset (MNIST)
train_dataset = datasets.MNIST(
    root="./data",
    train=True,
    transform=transform,
    download=True
)


100%|██████████| 9.91M/9.91M [00:00<00:00, 17.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 484kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.43MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.03MB/s]


In [5]:
#load test dataset
test_dataset = datasets.MNIST(
    root="./data",
    train=False,
    transform=transform,
    download=True
)


In [6]:
# Make DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# TODO: Access and print the unique labels in the training data set using the train_loader object
all_labels = []
for _, labels in train_loader:
    all_labels.append(labels)

#concat all label tensors and find unique values
all_labels = torch.cat(all_labels)
unique_labels = torch.unique(all_labels)
print(f"Unique labels in training data: {unique_labels}")

Unique labels in training data: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [7]:
import torch
import torch.nn as nn

# -----------------------------
# 2. DEFINE NEURAL NETWORK
# TODO: Design a Neural Network with 1 hidden layer of 128 neurons
# -----------------------------
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Define layers
        self.fc1 = nn.Linear(28*28, 128) #input layer (28*28 pixels) to hidden layer (128 neurons)
        self.fc2 = nn.Linear(128, 64)    #second hidden layer (64 neurons)
        self.fc3 = nn.Linear(64, 10)     #hidden layer (64 neurons) to output layer (10 classes for digits 0-9)

    def forward(self, x):
        #flatten image: (batch, 1, 28, 28) → (batch, 784)
        x = x.view(-1, 28*28)

        #activation between layers
        x = self.fc1(x)
        x = torch.relu(x) #apply relu activation to the first hidden layer output

        x = self.fc2(x)
        x = torch.relu(x) #apply relu activation to the second hidden layer output

        #output layer
        x = self.fc3(x)

        return x

In [8]:
# Create the model
model = SimpleNN()

In [9]:
# -----------------------------
# 3. LOSS FUNCTION + OPTIMIZER
# -----------------------------
# Define your loss function
loss_fn = nn.CrossEntropyLoss()

# Setup your gradient descent. Try different values for the learning rate
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# -----------------------------
# 4. TRAINING LOOP
# -----------------------------

# TODO: Define the number of epochs
epochs = 50

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        # TODO: Reset the gradients
        optimizer.zero_grad()

        # TODO: Forward pass
        outputs = model(images)

        # TODO: Compute loss
        loss = loss_fn(outputs, labels)

        # TODO: Backpropagate
        loss.backward()

        # TODO: Update gradients
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 311.8681
Epoch 2, Loss: 124.7659
Epoch 3, Loss: 86.4246
Epoch 4, Loss: 65.9047
Epoch 5, Loss: 50.3593
Epoch 6, Loss: 39.8354
Epoch 7, Loss: 32.5688
Epoch 8, Loss: 28.7505
Epoch 9, Loss: 20.4879
Epoch 10, Loss: 18.9052
Epoch 11, Loss: 16.6341
Epoch 12, Loss: 14.1064
Epoch 13, Loss: 13.7548
Epoch 14, Loss: 11.2296
Epoch 15, Loss: 11.1039
Epoch 16, Loss: 9.6772
Epoch 17, Loss: 10.4115
Epoch 18, Loss: 6.5433
Epoch 19, Loss: 8.1938
Epoch 20, Loss: 9.7030
Epoch 21, Loss: 8.4786
Epoch 22, Loss: 6.6661
Epoch 23, Loss: 7.9394
Epoch 24, Loss: 5.1979
Epoch 25, Loss: 6.6536
Epoch 26, Loss: 6.4011
Epoch 27, Loss: 8.1270
Epoch 28, Loss: 5.6159
Epoch 29, Loss: 4.9380
Epoch 30, Loss: 4.5887
Epoch 31, Loss: 7.1289
Epoch 32, Loss: 5.0846
Epoch 33, Loss: 6.8354
Epoch 34, Loss: 4.2686
Epoch 35, Loss: 6.4494
Epoch 36, Loss: 4.8251
Epoch 37, Loss: 3.1965
Epoch 38, Loss: 3.3918
Epoch 39, Loss: 6.9236
Epoch 40, Loss: 7.8110
Epoch 41, Loss: 2.3625
Epoch 42, Loss: 4.9914
Epoch 43, Loss: 4.1880
Ep

In [11]:
# -----------------------------
# 5. EVALUATION
# -----------------------------
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        # TODO: Forward pass
        outputs = model(images)

        # Predicted class = index of max logit
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 97.97%


In [12]:
# -----------------------------
# 6. TEST SINGLE PREDICTION
# -----------------------------
# -----------------------------
# 6. TEST SINGLE PREDICTION
# -----------------------------
# ------------------------------
# Gradio Sketchpad gives you:

# * a full-color NumPy array

# * black digit on white background

# * large resolution

# * no consistent scale
#
# Hence the preprocessing
# ------------------------------
import gradio as gr
from PIL import ImageOps

def preprocess_image(image):
    sketch_transform = transforms.Compose([
    transforms.ToPILImage(),                      #mumPy → PIL
    transforms.Grayscale(),                       #ensure 1 channel
    transforms.Resize((28, 28)),                  #28x28 like MNIST
    transforms.Lambda(lambda img: ImageOps.invert(img)),  # invert colors
    transforms.ToTensor(),                        # → tensor, shape (1,28,28), values in [0,1]
    ])

    if isinstance(image, dict):
        image = image['composite']


    img_tensor = sketch_transform(image)  # (1, 28, 28)


    img_tensor = img_tensor.unsqueeze(0)

    return img_tensor

def predict_digit(image):
    # --- STEP 1: CHECK IF SOMETHING HAS BEEN DRAWN ---
    if image is None: return "Draw something!"

    # --- STEP 2: PREPROCESS THE IMAGE ---
    img_tensor = preprocess_image(image)

    # --- STEP 3: RUN THE MODEL ---
    with torch.no_grad():
        prediction = model(img_tensor)


        predicted_digit = torch.argmax(prediction).item()

    return str(predicted_digit)

#UI Setup
interface = gr.Interface(fn=predict_digit, inputs=gr.Sketchpad(label="Draw Here"), outputs="label")
interface.queue().launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://869383b2a6b83119bc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
